In [1]:
import pandas as pd
# import tensorflow as tf
# import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
# import bert
# from bert import run_classifier
# from bert import optimization
# from bert import tokenization
# from tensorflow import keras
import os
import re
from datetime import datetime
import datasets
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TrainingArguments, Trainer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import evaluate
import numpy as np
from sklearn.preprocessing import LabelEncoder

/home/stud/t/ts218/miniconda3/envs/tf_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-10 11:41:12.127736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# create training dataset
data_files = {"train": "train.csv", "validation": "validation.csv", "test": "test.csv"}
dataset = datasets.load_dataset("./data/raw", data_files=data_files)

# tokenize dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_dataset(data):
    return tokenizer(data['description'])

dataset = dataset.map(tokenize_dataset)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 35.05it/s]


Dataset csv downloaded and prepared to /home/stud/t/ts218/.cache/huggingface/datasets/csv/raw-7d028b0f05d4348c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 162.42it/s]


In [3]:
# Load the model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=1391)

tf_train = model.prepare_tf_dataset(dataset['train'],batch_size=10, shuffle=True, tokenizer=tokenizer)
tf_val = model.prepare_tf_dataset(dataset['validation'],batch_size=10, shuffle=True, tokenizer=tokenizer)
tf_test = model.prepare_tf_dataset(dataset['test'],batch_size=10, shuffle=True, tokenizer=tokenizer)

print('resulting batches in each dataset:')
print(f'train: {tf_train.cardinality().numpy()}\nvalidate: {tf_val.cardinality().numpy()}\ntest: {tf_test.cardinality().numpy()}')

2023-05-10 11:41:34.417539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 11:41:36.853559: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11447 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:09:00.0, compute capability: 6.1
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerF

resulting batches in each dataset:
train: 340
validate: 37
test: 42


In [4]:
# configure callbacks
# checkpoint
filepath="./output/raw/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max', save_weights_only=True)
callbacks_list = [checkpoint]

# train model
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5), metrics=['accuracy'])
model.fit(tf_train, validation_data=tf_val, callbacks=callbacks_list, epochs=5)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/5
340/340 [==============================] - 60s 135ms/step - loss: 6.3981 - accuracy: 0.0994 - val_loss: 5.8545 - val_accuracy: 0.1676
Epoch 2/5
340/340 [==============================] - 45s 133ms/step - loss: 5.5304 - accuracy: 0.1812 - val_loss: 5.5226 - val_accuracy: 0.2297
Epoch 3/5
340/340 [==============================] - 46s 135ms/step - loss: 5.0858 - accuracy: 0.2312 - val_loss: 5.3924 - val_accuracy: 0.2649
Epoch 4/5
340/340 [==============================] - 47s 137ms/step - loss: 4.6522 - accuracy: 0.2976 - val_loss: 5.3970 - val_accuracy: 0.2676
Epoch 5/5
340/340 [==============================] - 47s 138ms/step - loss: 4.2855 - accuracy: 0.3335 - val_loss: 5.2508 - val_accuracy: 0.2892


In [7]:
model.save('./output/raw/test_model')

INFO:tensorflow:Assets written to: ./output/raw/test_model/assets


INFO:tensorflow:Assets written to: ./output/raw/test_model/assets
